In [1]:
# from src.open_r1.trl_main.trl.trainer import SFTTrainer
from trl import SFTTrainer
from datasets import load_dataset, Dataset
import json
import pandas as pd
import pyarrow.parquet as pq
from swanlab.integration.huggingface import SwanLabCallback
from peft import LoraConfig, TaskType, get_peft_model
import pyarrow.parquet as pq
import pandas as pd
from math_verify import LatexExtractionConfig, parse, verify
from latex2sympy2_extended import NormalizationConfig
import re
# 该代码多卡SFT 设置 cuda visiable 
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

/home/vipuser/miniconda3/envs/openr1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-06-02 13:59:28,622] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/vipuser/miniconda3/envs/openr1/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/vipuser/miniconda3/envs/openr1/compiler_compat/ld: warning: librt.so.1, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/vipuser/miniconda3/envs/openr1/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/vipuser/miniconda3/envs/openr1/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/vipuser/miniconda3/envs/openr1/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/vipuser/miniconda3/envs/openr1/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/home/vi

In [2]:
import pandas as pd
file_path = "../GRPO_data/training_prompt2.csv"
# model_path = "/home/LLM/models/Qwen2.5-0.5B-Instruct"
# model_path = "/home/LLM/models/Qwen2.5-0.5B-Instruct"
model_path = "/home/LLM/models/Qwen2.5-1.5B-Instruct"

df = pd.read_csv(file_path)

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model = AutoModelForCausalLM.from_pretrained(model_path, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(model_path)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [7]:
# new_df = df.head(10)
# print(len(df))
print(df)

                   db_id                                           question  \
0               academic  return me the conference, which has the most n...   
1                 bike_1  What are the dates with a maximum temperature ...   
2           storm_record  What are the names of storms that both affecte...   
3     student_assessment  What is detail of the student who most recentl...   
4               academic  return me the number of papers written by " H....   
...                  ...                                                ...   
8524            academic     return me all the keywords in Databases area .   
8525            flight_1  What are the names of all employees who can fl...   
8526                hr_1  display the full name (first and last), hire d...   
8527           allergy_1  What are the students ids of students who have...   
8528           chinook_1  Find the title of all the albums of the artist...   

                                                  q

In [8]:
system_prompt = """
You are a Schema linking assistant. Your task is to select appropriate table names and corresponding column names from the database in <database>...</database> based on the user's question in <question>...</question>. 
Follow this response template:\n<answer>\n###Tables: (Related Table);\n###Columns: (Related Columns from Tables);\n</answer> 
"""
print(system_prompt)


You are a Schema linking assistant. Your task is to select appropriate table names and corresponding column names from the database in <database>...</database> based on the user's question in <question>...</question>. 
Follow this response template:
<answer>
###Tables: (Related Table);
###Columns: (Related Columns from Tables);
</answer> 



In [9]:
def make_user_prompt(question, database_schema):
    return f"<question>{question}</question>\n<database>{database_schema}</database>"

def make_reponse(target_schema):
    return f"<answer>\n{target_schema}</answer>"

In [10]:
def make_conversation(question, database_schema,target_schema):
    conversation = []
    conversation0 = {}
    conversation1 = {}
    conversation2 = {}
    user_prompt = make_user_prompt(question, database_schema)
    assistance_prompt = make_reponse(target_schema)
    conversation0["role"] = "system"
    conversation0["content"] = system_prompt
    conversation1["role"] = "user"
    conversation1["content"] = user_prompt + "\n"
    conversation2["role"] = "assistant"
    conversation2["content"] = assistance_prompt
    conversation.append(conversation0)
    conversation.append(conversation1)
    conversation.append(conversation2)
    all_info = system_prompt + user_prompt + assistance_prompt
#     print(conversation2["content"])
    return conversation, all_info

In [11]:
trans_data = {}
message = []
max_token = 0
for index, row in df.iterrows():
    conversation, all_info = make_conversation(row["question"],row["database_schema"],row["target_schema"])
    message.append(conversation)
    token_len = len(tokenizer(all_info, add_special_tokens=False)["input_ids"])
#     print(all_info)
    if  token_len >  max_token:
        max_token = token_len
trans_data["messages"] = message

In [12]:
print(max_token)

2169


In [13]:
trans_data = pd.DataFrame(trans_data)
trans_data = Dataset.from_pandas(trans_data,split="train")
for sample in trans_data:
    print(sample)
    break  # 仅打印第一个样本

{'messages': [{'content': "\nYou are a Schema linking assistant. Your task is to select appropriate table names and corresponding column names from the database in <database>...</database> based on the user's question in <question>...</question>. \nFollow this response template:\n<answer>\n###Tables: (Related Table);\n###Columns: (Related Columns from Tables);\n</answer> \n", 'role': 'system'}, {'content': '<question>return me the conference, which has the most number of papers containing keyword " Relational Database " .</question>\n<database>CREATE TABLE `author` (\n  aid INT PRIMARY KEY,\n  homepage TEXT,\n  name TEXT,\n  oid INT\n);\n\nCREATE TABLE `conference` (\n  cid INT PRIMARY KEY,\n  homepage TEXT,\n  name TEXT\n);\n\nCREATE TABLE `domain` (\n  did INT PRIMARY KEY,\n  name TEXT\n);\n\nCREATE TABLE `domain_author` (\n  aid INT PRIMARY KEY REFERENCES author(aid),\n  did INT PRIMARY KEY REFERENCES domain(did)\n);\n\nCREATE TABLE `domain_conference` (\n  cid INT PRIMARY KEY REFER

In [15]:
output_dir = "../SFT_only_model-1.5B"
num_train_epochs = 6
bf16 = True
overwrite_output_dir = True
per_device_train_batch_size = 4
gradient_accumulation_steps = 8
gradient_checkpointing = True
evaluation_strategy = "steps"
learning_rate = 5e-5
weight_decay = 0.01
lr_scheduler_type = "cosine"
warmup_ratio = 0.01
max_grad_norm = 0.3
group_by_length = True
auto_find_batch_size = False
save_steps = 10
logging_steps = 10
load_best_model_at_end= False
packing = False
save_total_limit=1
neftune_noise_alpha=5
# report_to="wandb"

In [16]:
from transformers import TrainingArguments
from swanlab.integration.huggingface import SwanLabCallback
training_arguments = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    num_train_epochs=num_train_epochs,
    load_best_model_at_end=load_best_model_at_end,
    per_device_train_batch_size=per_device_train_batch_size,
#     evaluation_strategy=evaluation_strategy,
    max_grad_norm = max_grad_norm,
    auto_find_batch_size = auto_find_batch_size,
    save_total_limit = save_total_limit,
    gradient_accumulation_steps=gradient_accumulation_steps,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    bf16=bf16,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="none",
    neftune_noise_alpha= neftune_noise_alpha
)

In [17]:
# swanlab_callback = SwanLabCallback(
#     project="Qwen2.5_1.5B_GRPO_steer_CoT",
#     experiment_name="Qwen2.5_1.5B_GRPO_steer_CoT",
#     description="Qwen2.5_1.5B_GRPO_steer_CoT",
#     config={
#         "model": "Qwen2.5-1.5B",
#         "dataset": "steer_CoT",
#     },
# )

In [18]:
trainer = SFTTrainer(
    model= model,
    train_dataset=trans_data,
#     peft_config = peft_config,
    args=training_arguments,
#     callbacks=[swanlab_callback],
#     params_mask = params_mask
)

# trainer.train()

Truncating train dataset: 100%|███████████████████████████████████████████████████████████████████| 8529/8529 [00:00<00:00, 60718.82 examples/s]


In [19]:
trainer.train()

Step,Training Loss
10,0.840000
20,0.403800
30,0.445200
40,0.553000
50,0.406200
60,0.606800
70,0.222200
80,0.219900
90,0.231900
100,0.205000


TrainOutput(global_step=1596, training_loss=0.09227143763039346, metrics={'train_runtime': 8197.1086, 'train_samples_per_second': 6.243, 'train_steps_per_second': 0.195, 'total_flos': 1.6938471137139917e+17, 'train_loss': 0.09227143763039346})

In [20]:
trainer.model.save_pretrained(output_dir)

In [21]:
import os
import shutil
from pathlib import Path
 
# 源文件路径
tokenizer_config_path = os.path.join(model_path, "tokenizer_config.json")
tokenizer_path = os.path.join(model_path, "tokenizer.json")
  
# 确保目标目录存在
Path(output_dir).mkdir(parents=True, exist_ok=True)
 
# 复制文件
try:
    shutil.copy2(tokenizer_config_path, output_dir)
    shutil.copy2(tokenizer_path, output_dir)
    print("文件复制成功！")
except FileNotFoundError as e:
    print(f"错误：找不到源文件 - {e}")
except Exception as e:
    print(f"发生错误：{e}")# inference 

文件复制成功！


In [ ]:
#  use_my_SFTtrainer 自己的trl库源码
# processing_class, train_dataset = trainer.get_dataset()
# print(train_dataset)
# print(processing_class)
# # completions_text = self.processing_class.batch_decode(completion_ids, skip_special_tokens=True)

In [16]:
# print(type(train_dataset))

<class 'datasets.arrow_dataset.Dataset'>


In [17]:
# for i in train_dataset:
#     print(i)
#     break

{'source': 'aplaca', 'num_turns': 2, 'text': "<|im_start|>system\nYou are a helpful AI Assistant that provides well-reasoned and detailed responses. You first think about the reasoning process as an internal monologue and then provide the user with the answer. Respond in the following format: <think>\n...\n</think>\n<answer>\n...\n</answer><|im_end|>\n<|im_start|>user\nDescribe a process of making crepes.\n<|im_end|>\n<|im_start|>assistant\nMaking crepes is an easy and delicious process! Here are step-by-step instructions on how to make them:\n\n1. Assemble your ingredients. For basic crepes, you'll need: 1 cup all-purpose flour, 2 eggs, 1/2 cup milk, 1/2 cup water, 1/4 teaspoon salt, and 2 tablespoons melted butter.\n\n2. Mix the batter: In a large mixing bowl, whisk together the flour and the eggs. Gradually add the milk and water, stirring constantly to ensure that there are no lumps. Add salt and melted butter, and mix well.\n\n3. Let the batter rest: If you can, let the batter sit

In [14]:
# trainer.train()

swanlab: swanlab version 0.5.5 is available!  Upgrade: `pip install -U swanlab`    
swanlab: Tracking run with swanlab version 0.5.4                                   
swanlab: Run data will be saved locally in /home/LLM/code/open-r1-main/swanlog/run-20250409_133541-a3b1799d
swanlab: 👋 Hi winhong, welcome to swanlab!
swanlab: Syncing run Qwen2.5-1.5B_Full_Aplaca_test_GRPO_SFT to the cloud
swanlab: 🏠 View project at https://swanlab.cn/@winhong/Qwen2.5-1.5B_Full_Aplaca_test_GRPO_SFT
swanlab: 🚀 View run at https://swanlab.cn/@winhong/Qwen2.5-1.5B_Full_Aplaca_test_GRPO_SFT/runs/wjtnv2yua7qkfkwpa4cmg


KeyboardInterrupt: 

In [20]:
output_dir = os.path.join("./", "/home/LLM/models/SFT_model/Qwen_1.5B_GRPO_test")
trainer.model.save_pretrained(output_dir)